In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
# rutor_url = 'http://5.45.84.133/new/'
rutor_url = 'http://rutor.info/new'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)]
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
5,Зарубежные фильмы,02 Июн 23,Гипнотик / Hypnotic (2023) WEB-DLRip-AVC от DoMiNo & селезень | P | HDRezka Studio,1.37 GB,59,53
8,Зарубежные фильмы,02 Июн 23,Гипнотик / Hypnotic (2023) WEB-DLRip 720p от DoMiNo & селезень | P | HDRezka Studio,2.15 GB,60,43
12,Зарубежные фильмы,02 Июн 23,"Мизантроп / Misanthrope / To Catch A Killer (2023) WEB-DLRip-AVC от DoMiNo & селезень | D, P",1.90 GB,44,17
17,Зарубежные фильмы,02 Июн 23,Гипнотик / Hypnotic (2023) WEB-DLRip-AVC от ExKinoRay | HDRezka Studio,1.26 GB,161,34
19,Зарубежные фильмы,02 Июн 23,Ночная сова / Olppaemi / The Night Owl (2022) WEB-DLRip от MegaPeer | D,1.46 GB,59,13
20,Зарубежные фильмы,02 Июн 23,Гипнотик / Hypnotic (2023) WEB-DL 1080p от селезень | P,4.90 GB,372,152
21,Зарубежные фильмы,02 Июн 23,Мизантроп / Misanthrope / To Catch A Killer (2023) WEB-DLRip-AVC от ExKinoRay | D,1.78 GB,41,22
22,Зарубежные фильмы,02 Июн 23,Реалити / Reality (2023) WEB-DLRip от MegaPeer | HDRezka Studio,1.47 GB,58,25
23,Зарубежные фильмы,02 Июн 23,Гипнотик / Hypnotic (2023) WEB-DL 720p от ExKinoRay | HDRezka Studio,2.53 GB,243,100
24,Зарубежные фильмы,02 Июн 23,Гипнотик / Hypnotic (2023) WEB-DLRip-AVC | HDRezka Studio,1.59 GB,100,12


In [9]:
id = [90]
my_df.loc[id, 'href']

90    http://rutor.info/torrent/922430/ljubov-i-smert_love-death-s01-2023-web-dlrip-newstudio
Name: href, dtype: object